En este notebook se busca extraer la información necesaria para ser consumida por los modelos.

Variables a considerar:
- Aportes hídricos
- Compras en bolsa
- Compras en contratos
- Demanda del SIN
- Disponibilidad Declarada
- Disponibilidad Real
- Generación real
- Indice ONI
- Precio de bolsa
- Precio de oferta
- Reservas hídricas
- Ventas en bolsa
- Ventas en contratos
- Vertimientos


In [8]:
!pip install pydataxm
# !pip list

In [9]:
import datetime as dt
import pydataxm
from pydataxm.pydataxm import ReadDB
import pandas as pd
import pathlib
import os
import time
# os.chdir(pathlib.Path.cwd())

#Conexión de google drive con el repo
from google.colab import drive
drive.mount('/content/drive')

###==========================
ruta = os.path.join("/content/drive/MyDrive", "MonografiaUdeA")
os.chdir(ruta)
# print("Ruta:", ruta)
abspath = pathlib.Path.cwd()
print(f"Directorio de trabajo: {abspath}")
from periods import get_periods
###==========================

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directorio de trabajo: /content/drive/MyDrive/MonografiaUdeA


In [10]:
# !git status

In [11]:
consult = ReadDB()
consult.get_collections().sort_values(by = 'MetricName').to_csv(os.path.join(abspath, "metrics.csv"), index=False)

## Parámetros para la extracción de la información

In [12]:
start_date = dt.date(2010, 1, 1)
# end_date = dt.date(2000,1,31)
end_date = dt.date(2019, 12, 31)

months_per_period = 12
periods = get_periods(start_date, end_date, months_per_period)


dataset_folder_name = "datasets"
metricas = [
    # ["AporEner","Sistema"],
    # ["CompBolsNaciEner", "Sistema"],
    # ["CompContEner", "Sistema"],
    # ["DemaReal","Sistema"],
    # ["DispoDeclarada", "Recurso"],
    # ["DispoReal", "Recurso"],
    # ["Gene", "Recurso"],
    # ["PrecBolsNaci", "Sistema"],
    # ["PrecOferDesp", "Recurso"],
    # ["VentBolsNaciEner", "Sistema"],
    # ["VentContEner", "Sistema"],
    # ["VoluUtilDiarEner", "Sistema"],
    # ["VertEner", "Sistema"]
]

## Obtener listado de los recursos de generación

In [13]:
def obtener_recursos():
  recursos_generacion = consult.request_data(
                                            "ListadoRecursos",
                                            "Sistema",
                                            start_date,
                                            end_date
                                            )

  recursos_generacion.to_csv(os.path.join(abspath, dataset_folder_name, "listado_recursos.csv"), index=False)
  recursos_generacion.head()

# obtener_recursos()

## Obtener las demás variables

In [14]:
if not os.path.exists(os.path.join(abspath, dataset_folder_name)):
    os.makedirs(os.path.join(abspath, dataset_folder_name))

def extraer():
  for metrica in metricas:
      consult = ReadDB()
      df = pd.DataFrame()
      print(metrica)
      # metrica = _reg
      for period in periods:
          print(period[0], period[1])
          df_cons = consult.request_data(metrica[0], metrica[1], period[0], period[1])
          df = pd.concat([df, df_cons])
          time.sleep(2)

      df.to_csv(os.path.join(abspath, dataset_folder_name, f"{metrica[0]}.csv"), index=False)

extraer()